In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from math import ceil
import pickle

import numpy as np
import pandas as pd

In [2]:
datestring_old = '20191118'
datestring_new = '20200210'

In [3]:
# nacitaj relativne linky ku vsetkym konaniam
with open('./Data/rel_linky_list_'+datestring_old+'.pkl', 'rb') as f:
    relativne_linky_old = pickle.load(f)
    
with open('./Data/rel_linky_list_'+datestring_new+'.pkl', 'rb') as f:
    relativne_linky = pickle.load(f)

In [4]:
# vytiahni indexy z relativnych linkov
def get_indexy_konani(rel_link):
    return rel_link.split('/')[-1]

indexy_konani_old = list(map(get_indexy_konani, relativne_linky_old))
indexy_konani = list(map(get_indexy_konani, relativne_linky))

# identifikuj nove vyhlasene konania
indexy_konani_new = np.setdiff1d(indexy_konani, indexy_konani_old)
print(indexy_konani_new.size)

1206


In [5]:
# scraping treba zbehnut aj na predtym preskocenych vyberovych konaniach, 
# ktore mohli byt aktualizovane (vyhlasenie vysledku)

with open('./Data/log_preskocene_'+datestring_old+'.txt') as f:
    preskocene = f.readlines()

# zbehnutie kodu ak zlyha ukladanie niektorych datasetov
with open('./Data/log_preskocene_chyby_'+datestring_old+'.txt') as f:
    preskocene = preskocene + f.readlines()

In [6]:
# transformuj do formatu index - (priebežný) výsledok konania
def get_index_result(preskoceny_zaznam):
    preskoceny_zaznam_list = preskoceny_zaznam.split(' ')
    index = int(preskoceny_zaznam_list[1])
    vysledok = ' '.join(preskoceny_zaznam_list[3:]).strip('\n')
    
    return (index, vysledok)

preskocene_list = list(map(get_index_result, preskocene))
print(len(preskocene_list))

# poradime si s duplikatmi (opakovany zapis zaznamov, ktore sa kvoli chybe neulozili ale neskor sme ich znova zbehli)
df_preskocene_old = pd.DataFrame(pd.Series(dict(preskocene_list)))
df_preskocene_old.index.name = 'index_konania'
df_preskocene_old.rename(columns={0:'vysledok'}, inplace=True)
print('Unikatnych', df_preskocene_old.index.size)

1546
Unikatnych 1340


In [7]:
preskocene_list = list(df_preskocene_old.index.astype(str))
indexy_konani_new = list(indexy_konani_new)
# odstran priapdne duplicity
indexy_konani_update = list(set(indexy_konani_new + preskocene_list))

In [8]:
# skontroluj duplicity

print(len(preskocene_list) + len(indexy_konani_new))
print(len(indexy_konani_update))

print('Duplicity: ', (len(preskocene_list) + len(indexy_konani_new))!=len(indexy_konani_update))

2546
2546
Duplicity:  False


In [11]:
# uloz list relativnych linkov
with open('./Data/indexy_konani_list_'+datestring_new+'.pkl', 'wb') as f:
    pickle.dump(indexy_konani_update, f)